In [1]:
import logging

from Loader import fileloader_proto as fl
from Loader import dfutils
import xlwings as xw
import pandas as pd
import numpy as np
import posixpath

logger = logging.getLogger("")
logger.setLevel(logging.INFO)

month = '201903'

inifile = fl.ReadIniFile(mercado="empresas")
#defaultpath = inifile.getDataPath()
#testpath = inifile.getTestPath()
#parser = inifile.getIniFileParser()


**Procedimiento**
---

- Copiar archivos de comisiones a la carpeta de reporte de productividad
- Ejecutar Script para normalizar pesos
- En el archivo final de R. Productividad eliminar los cesados y agregar los nuevos.
- Ejecutar Script para cuadrar "reporte de productividad". Ojo: Este usa el orden de planilla que esta en el reporte final de productividad.
- Este ultimo script creara un archivo en testpath, usar ese para copiar y pegar las columnas en el reporte final de productividad
    

**Normalizar Matriz de Pesos de Comisiones**
---

Tomamos $A_{mxn}=\begin{bmatrix}a_{i,j}\end{bmatrix}$y queremos obtener
$\begin{bmatrix}\frac{a_{i,j}}{\sum_{k=1}^{n}a_{i,k}}\end{bmatrix}_{1xm}$

Hallamos el vector suma de las filas:

$A_{s}=\begin{bmatrix}\sum_{k=1}^{n}a_{i,k}\end{bmatrix}_{1xm}$

Invertimos:

$(A_{s}^{-1})_{1xm}$

Hallamos la diagonal:

\{$diag(A_{s}^{-1})\}_{mxm}$

Multiplicamos esta matriz diagonal por la matriz inicial para obtener
lo buscado:

\{$diag(A_{s}^{-1})\}_{mxm}*A_{mxn}=T\{a_{i,j}\}$

$\begin{bmatrix}*_{1}\\
 & *_{2}\\
 &  & \ddots
\end{bmatrix}*\begin{bmatrix}\Delta_{1,1} & \Delta_{1,2}\begin{array}{c}
\cdots\end{array}\\
\Delta_{2,1} & \Delta_{2,2}\begin{array}{c}
\cdots\end{array}\\
\vdots & \vdots
\end{bmatrix}=\begin{bmatrix}*_{1}\Delta_{1,1} & *_{1}\Delta_{1,2}\begin{array}{c}
\cdots\end{array}\\
*_{2}\Delta_{2,1} & *_{2}\Delta_{2,2}\begin{array}{c}
\cdots\end{array}\\
\vdots & \vdots
\end{bmatrix}=T\{a_{i,j}\}$

In [5]:
#Escoger que archivo se desea normalizar los pesos
#chosen_file = "Productividad_Plataformas_Comerciales"
#chosen_file = "Productividad_Grandes_Cuentas"
#chosen_file = "Productividad_Soluciones_Negocio"
#chosen_file = "Productividad_Pymes"

# Dataframe de archivo comisiones pestaña "Leyenda" de la ruta de Reporte de Productividad
section_1 = fl.SectionObj(inifile,chosen_file,month)
section_1.setParameter('presetsheet','Leyenda')
loader1 = fl.LoadFileProcess(section_1)
pesospltfrs = loader1.loadFile()

# Dataframe de archivo comisiones pestaña "Comisionantes" de la ruta de Reporte de Productividad
section_2 = fl.SectionObj(inifile,chosen_file,month)
section_2.setParameter('presetsheet','Comisionantes')
loader2 = fl.LoadFileProcess(section_2)
comisionantespltfrs = loader2.loadFile()

#Obteniendo ruta del archivo de comisiones para ser accedida por XlWings
file = section_2.getParameter('filelist')[0]
logger.debug(file)

#Abriendo archivo de Excel y haciendo hoja 'comisionantes' activa
wb = xw.Book(file)
comis_sheet = wb.sheets('Comisionantes')
leyenda_sheet = wb.sheets('Leyenda')
#for sheet in wb.sheets:
#  if 'Leyenda' in sheet:
#    logger.debug("yes")

### Inmovilizando valores de porcentajes ponderados
#Obteniendo indice de la columna de porcentaje ponderado
ponderado_cindex = dfutils.getExcelColIndexFromDF(comisionantespltfrs, "PORCENTAJE_TOTAL_PONDERADO")
#Hallando ultima fila de la columna 'PORCENTAJE_TOTAL_PONDERADO'
lastrow = comis_sheet.api.Cells(65536, ponderado_cindex).End(xw.constants.Direction.xlUp).Row
#Copiando como valores las contenidos de la columna
comis_sheet.range((1, ponderado_cindex)).options(transpose=True).value = comis_sheet.range((1, ponderado_cindex), (lastrow,ponderado_cindex)).value

# Nombres clave de la hoja leyenda, siempre verificar que esto no cambie, en caso lo haga modificar la siguiente tupla. Porque de lo contrario el algoritmo dara errores / no funcionara como se espera.
COMIS_COLUMNS_NAMES = ("CAPTURA", "GESTIÓN", "DESARROLLO", "ITEM")

### Filtramos la data de la pestaña 'Leyenda' y nos quedamos con la sección de pesos únicamente en el dataframe pesos_df
pesos_df = pesospltfrs
pesos_df = pesos_df[pesos_df['ITEM'] != COMIS_COLUMNS_NAMES[3]]
pesos_df = pesos_df[~pesos_df['ITEM'].isnull()]
pesos_df = pesos_df[pesos_df['ITEM'] < 3000]
pesos_df = pesos_df[pesos_df['ITEM'] > 2000]

mat_result = []

for i in range(3):
    regexp = COMIS_COLUMNS_NAMES[i] + ".*"
    #Nos quedamos unicamente con las columnas que comienzan con el patron en 'regexp'
    area_df = pesos_df.filter(regex=regexp)
    #Guardamos las cabeceras
    headers = area_df.columns.values
    area_df = area_df.fillna(0) #fill empty spaces read as nan to zeros
    #Convertimos el dataframe en una matriz numpy (array representation)
    area_mat = area_df.as_matrix() #mxn
    #Operaciones de matrices
    sum_mat_1 = np.sum(area_mat, axis=1) #1xm, sum all the rows
    #Invertimos escalarmente el vector
    sum_mat_2 = 1 / sum_mat_1
    #Reemplazamos los NaN por ceros
    sum_mat = np.nan_to_num(sum_mat_2, copy=True)
    #Creamos una matriz diagonal, donde los elementos en la diagonal son las sumas de cada fila.
    diag_mat = np.diag(sum_mat) #mxm
    #Multiplicacion matricial entre la matriz diagonal y la matrix original
    t_mat = np.dot(diag_mat, area_mat) #mxm x mxn = mxn
    #Resultado buscado
    df_conv = pd.DataFrame(t_mat, columns=headers)
    mat_result.append(df_conv)

df_conv = pd.concat([mat_result[0], mat_result[1], mat_result[2]], axis=1)

# Creamos copia de seguridad de la tabla de pesos antes de modificarla.
wb.sheets.add('backup_pesos')
backup_pesos_sheet = wb.sheets('backup_pesos')
backup_pesos_sheet.range('A1').value = pesos_df

#Buscar numero de columna CAPTURA 1
col_pesos = pesospltfrs.columns.get_loc(COMIS_COLUMNS_NAMES[0] + "_1") + 1
#Buscar  numero de fila primera ocurrencia de ITEM
row_pesos = min(pesospltfrs.index[pesospltfrs['ITEM'] == COMIS_COLUMNS_NAMES[3]].tolist()) + 4
# Escribimos los nuevos pesos en el lugar de los antiguos.
leyenda_sheet.range(row_pesos, col_pesos).options(pd.DataFrame, index=False).value = df_conv





El tamaño de Productividad_Pymes es 64 registros
El tamaño de Productividad_Pymes es 95 registros


C:\Users\Johnny\Anaconda3\lib\site-packages\ipykernel_launcher.py:59: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\Johnny\Anaconda3\lib\site-packages\ipykernel_launcher.py:63: RuntimeWarning: divide by zero encountered in true_divide


**Cuadrar Reporte Productividad**
---

In [3]:
# Cargar Data Frame de DNIs de los comisionantes del Reporte de Productividad Final
section_3 = fl.SectionObj(inifile,"Productividad_Final")
loader3 = fl.LoadFileProcess(section_3)
dnis_rp = loader3.loadFile()

section_4 = fl.SectionObj(inifile,"Padron_Empleados",month)
loader4 = fl.LoadFileProcess(section_4)
empleados_df = loader4.loadFile()
empleados_df = empleados_df[["DNI","JEFE_DIRECTO"]]

# Cargar Dataframe de archivos de pesos comisiones
section_5 = fl.SectionObj(inifile,"Productividad_All_Files",month)
loader5 = fl.LoadFileProcess(section_5)
planilla_comis= loader5.loadFile()
planilla_comis = planilla_comis[["GERENCIA2","DNI","NOMBRES","POSICIÓN","FECHA_DE_INGRESO","PORCENTAJE_TOTAL_PONDERADO","FACTOR_DE_PAGO","CAPTURA","GESTIÓN","DESARROLLO"]]

El tamaño de Productividad_Final es 450 registros
El tamaño de Padron_Empleados es 2819 registros


C:\Users\Johnny\Anaconda3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


El tamaño de Productividad_All_Files es 451 registros


In [4]:
# Cruzamos con DNI del archivo de R. Productividad final
df1 = pd.merge(dnis_rp, planilla_comis, on='DNI', how='left')
# Cruzamos con Ingresos para obtener el JEFE DIRECTO.
df2 = pd.merge(df1, empleados_df, on='DNI', how='left')
#Eliminamos duplicados, pueden darse porque ftocci por ejemplo
df2.drop_duplicates('DNI')
#Establecemos el orden de columnas de salida
df2 = df2[["GERENCIA2","DNI","NOMBRES","POSICIÓN","FECHA_DE_INGRESO","JEFE_DIRECTO","PORCENTAJE_TOTAL_PONDERADO","FACTOR_DE_PAGO","CAPTURA","GESTIÓN","DESARROLLO"]]




In [6]:
dataoutdir = inifile.getTestPath()
writer = pd.ExcelWriter(posixpath.join(dataoutdir,month + "_ReporteProdutividad.xlsx"), engine='xlsxwriter')
df2.to_excel(writer, sheet_name='Sheet1')
writer.save()